In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pathlib import Path

from attila.data.parse import parse_data, get_data
from attila.util.config import get_config
from attila.util.plots import plot_sample
from attila.experiments.data import load_experiments
from attila.experiments.do import do_batch_experiments
from attila.experiments.tools import create_tex_experiments

In [3]:
_here = Path('.').resolve()
config = get_config(_here / 'config.ini')

data_path = _here / config.get('data', 'folder')
data_path = data_path.resolve()

out_path = Path(config.get('experiments', 'output folder')).resolve()
out_path.mkdir(parents=True, exist_ok=True)  # rm and mkdir if existing

images_path = data_path / config.get('data', 'images')
masks_path = data_path / config.get('data', 'masks')

In [4]:
raw = get_data(images_path, masks_path)
X, y = parse_data(
    raw,
    (config.getint('image', 'width'), config.getint('image', 'height'))
)

In [5]:
experiments_file = _here / 'experiments.json'
experiments = load_experiments(experiments_file)

In [6]:
do_batch_experiments(experiments, (X, y), config, out_path)

testing data: X ~ (28, 256, 256, 1), y ~ (28, 256, 256, 3)
ready to perform #1 / 3 batch of experiments
augmented training data: X ~ (216, 256, 256, 1), y ~ (216, 256, 256, 3)
ready to perform 8 experiments
=== experiment #1 / 8: with_same
= dataset training: X ~ (216, 256, 256, 1), y ~ (216, 256, 256, 3)
= dataset validation: X ~ (10, 256, 256, 1), y ~ (10, 256, 256, 3)
= dataset test: X ~ (28, 256, 256, 1), y ~ (28, 256, 256, 3)
=== model
= # layers: 25
= # total params: 1882115
= # trainable params: 1882115
= # non-trainable params: 0
Epoch 1/10
53/54 [============================>.] - ETA: 0s - loss: 0.1009 - accuracy: 0.9592 - batch_metric-mean_IoU: 0.5035 - batch_metric-mean_DSC: 0.3917
Epoch 00001: loss improved from inf to 0.09945, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/with_same/model.h5
54/54 [==============================] - 4s 65ms/step - loss: 0.0994 - accuracy: 0.9597 - batch_metric-mean_IoU: 0.5056 - batch_metric-mean_DSC: 0.3934 - val_loss: 0.0149 -

Epoch 4/10
53/54 [============================>.] - ETA: 0s - loss: 0.0077 - accuracy: 0.9951 - batch_metric-mean_IoU: 0.7473 - batch_metric-mean_DSC: 0.5686
Epoch 00004: loss improved from 0.00916 to 0.00778, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/without_same/model.h5
54/54 [==============================] - 2s 43ms/step - loss: 0.0078 - accuracy: 0.9951 - batch_metric-mean_IoU: 0.7470 - batch_metric-mean_DSC: 0.5685 - val_loss: 0.0084 - val_accuracy: 0.9942 - val_batch_metric-mean_IoU: 0.7171 - val_batch_metric-mean_DSC: 0.5463 - lr: 0.0010
Epoch 5/10
53/54 [============================>.] - ETA: 0s - loss: 0.0059 - accuracy: 0.9964 - batch_metric-mean_IoU: 0.7869 - batch_metric-mean_DSC: 0.5920
Epoch 00005: loss improved from 0.00778 to 0.00594, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/without_same/model.h5
54/54 [==============================] - 2s 43ms/step - loss: 0.0059 - accuracy: 0.9964 - batch_metric-mean_IoU: 0.7871 - batch_metric-mean

Epoch 8/10
53/54 [============================>.] - ETA: 0s - loss: 0.0044 - accuracy: 0.9972 - batch_metric-mean_IoU: 0.8261 - batch_metric-mean_DSC: 0.6112
Epoch 00008: loss improved from 0.00542 to 0.00443, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/with_valid/model.h5
54/54 [==============================] - 2s 41ms/step - loss: 0.0044 - accuracy: 0.9972 - batch_metric-mean_IoU: 0.8264 - batch_metric-mean_DSC: 0.6113 - val_loss: 0.0056 - val_accuracy: 0.9965 - val_batch_metric-mean_IoU: 0.7967 - val_batch_metric-mean_DSC: 0.5960 - lr: 0.0010
Epoch 9/10
53/54 [============================>.] - ETA: 0s - loss: 0.0044 - accuracy: 0.9972 - batch_metric-mean_IoU: 0.8337 - batch_metric-mean_DSC: 0.6147
Epoch 00009: loss did not improve from 0.00443
54/54 [==============================] - 2s 39ms/step - loss: 0.0044 - accuracy: 0.9971 - batch_metric-mean_IoU: 0.8333 - batch_metric-mean_DSC: 0.6145 - val_loss: 0.0063 - val_accuracy: 0.9960 - val_batch_metric-mean_IoU: 0.79

54/54 [==============================] - ETA: 0s - loss: 0.1769 - accuracy: 0.9683 - batch_metric-mean_IoU: 0.4118 - batch_metric-mean_DSC: 0.3328
Epoch 00001: loss improved from inf to 0.17695, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/with_same_se/model.h5
54/54 [==============================] - 4s 74ms/step - loss: 0.1769 - accuracy: 0.9683 - batch_metric-mean_IoU: 0.4118 - batch_metric-mean_DSC: 0.3328 - val_loss: 0.0220 - val_accuracy: 0.9873 - val_batch_metric-mean_IoU: 0.5973 - val_batch_metric-mean_DSC: 0.4502 - lr: 0.0010
Epoch 2/10
54/54 [==============================] - ETA: 0s - loss: 0.0128 - accuracy: 0.9920 - batch_metric-mean_IoU: 0.6665 - batch_metric-mean_DSC: 0.5099
Epoch 00002: loss improved from 0.17695 to 0.01280, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/with_same_se/model.h5
54/54 [==============================] - 3s 58ms/step - loss: 0.0128 - accuracy: 0.9920 - batch_metric-mean_IoU: 0.6665 - batch_metric-mean_DSC: 0.5099 - 

54/54 [==============================] - 3s 51ms/step - loss: 0.0061 - accuracy: 0.9962 - batch_metric-mean_IoU: 0.7831 - batch_metric-mean_DSC: 0.5901 - val_loss: 0.0069 - val_accuracy: 0.9955 - val_batch_metric-mean_IoU: 0.7752 - val_batch_metric-mean_DSC: 0.5858 - lr: 0.0010
Epoch 5/10
53/54 [============================>.] - ETA: 0s - loss: 0.0062 - accuracy: 0.9961 - batch_metric-mean_IoU: 0.7851 - batch_metric-mean_DSC: 0.5913
Epoch 00005: loss did not improve from 0.00613
54/54 [==============================] - 3s 49ms/step - loss: 0.0062 - accuracy: 0.9961 - batch_metric-mean_IoU: 0.7853 - batch_metric-mean_DSC: 0.5914 - val_loss: 0.0060 - val_accuracy: 0.9962 - val_batch_metric-mean_IoU: 0.7797 - val_batch_metric-mean_DSC: 0.5874 - lr: 0.0010
Epoch 6/10
53/54 [============================>.] - ETA: 0s - loss: 0.0053 - accuracy: 0.9967 - batch_metric-mean_IoU: 0.8103 - batch_metric-mean_DSC: 0.6042
Epoch 00006: loss improved from 0.00613 to 0.00537, saving model to /home/h4/st

...

Epoch 7/10
53/54 [============================>.] - ETA: 0s - loss: 0.0053 - accuracy: 0.9967 - batch_metric-mean_IoU: 0.8049 - batch_metric-mean_DSC: 0.6011
Epoch 00007: loss improved from 0.00581 to 0.00528, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_same/model.h5
54/54 [==============================] - 2s 43ms/step - loss: 0.0053 - accuracy: 0.9967 - batch_metric-mean_IoU: 0.8053 - batch_metric-mean_DSC: 0.6013 - val_loss: 0.0070 - val_accuracy: 0.9953 - val_batch_metric-mean_IoU: 0.7930 - val_batch_metric-mean_DSC: 0.5965 - lr: 0.0010
Epoch 8/10
53/54 [============================>.] - ETA: 0s - loss: 0.0052 - accuracy: 0.9967 - batch_metric-mean_IoU: 0.8076 - batch_metric-mean_DSC: 0.6028
Epoch 00008: loss improved from 0.00528 to 0.00524, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_same/model.h5
54/54 [==============================] - 2s 43ms/step - loss: 0.0052 - accuracy: 0.9967 - batch_metric-mean_IoU: 0.8073 - batch_metric-mean

7/7 [==============================] - 0s 11ms/step
=== evaluation stats
= metrics on test set (size: 28)
= batch_metric-mean_IoU ~ mean 0.847 median 0.846 std 0.020
= batch_metric-mean_DSC ~ mean 0.621 median 0.620 std 0.008
=== experiment #4 / 8: without_valid
= dataset training: X ~ (216, 256, 256, 1), y ~ (216, 212, 212, 3)
= dataset validation: X ~ (10, 256, 256, 1), y ~ (10, 212, 212, 3)
= dataset test: X ~ (28, 256, 256, 1), y ~ (28, 212, 212, 3)
=== model
= # layers: 22
= # total params: 1697795
= # trainable params: 1697795
= # non-trainable params: 0
Epoch 1/10
53/54 [============================>.] - ETA: 0s - loss: 0.1290 - accuracy: 0.9575 - batch_metric-mean_IoU: 0.4412 - batch_metric-mean_DSC: 0.3536
Epoch 00001: loss improved from inf to 0.12696, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_valid/model.h5
54/54 [==============================] - 2s 44ms/step - loss: 0.1270 - accuracy: 0.9581 - batch_metric-mean_IoU: 0.4431 - batch_metric-mean_DSC: 

Epoch 4/10
54/54 [==============================] - ETA: 0s - loss: 0.0069 - accuracy: 0.9956 - batch_metric-mean_IoU: 0.7525 - batch_metric-mean_DSC: 0.5717
Epoch 00004: loss improved from 0.01027 to 0.00694, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/with_same_se/model.h5
54/54 [==============================] - 3s 59ms/step - loss: 0.0069 - accuracy: 0.9956 - batch_metric-mean_IoU: 0.7525 - batch_metric-mean_DSC: 0.5717 - val_loss: 0.0083 - val_accuracy: 0.9947 - val_batch_metric-mean_IoU: 0.7606 - val_batch_metric-mean_DSC: 0.5783 - lr: 0.0010
Epoch 5/10
54/54 [==============================] - ETA: 0s - loss: 0.0056 - accuracy: 0.9966 - batch_metric-mean_IoU: 0.7918 - batch_metric-mean_DSC: 0.5945
Epoch 00005: loss improved from 0.00694 to 0.00564, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/with_same_se/model.h5
54/54 [==============================] - 3s 59ms/step - loss: 0.0056 - accuracy: 0.9966 - batch_metric-mean_IoU: 0.7918 - batch_metric-mean

Epoch 8/10
53/54 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9969 - batch_metric-mean_IoU: 0.8081 - batch_metric-mean_DSC: 0.6031
Epoch 00008: loss improved from 0.00526 to 0.00505, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_same_se/model.h5
54/54 [==============================] - 3s 51ms/step - loss: 0.0051 - accuracy: 0.9969 - batch_metric-mean_IoU: 0.8084 - batch_metric-mean_DSC: 0.6032 - val_loss: 0.0064 - val_accuracy: 0.9959 - val_batch_metric-mean_IoU: 0.8223 - val_batch_metric-mean_DSC: 0.6106 - lr: 0.0010
Epoch 9/10
53/54 [============================>.] - ETA: 0s - loss: 0.0047 - accuracy: 0.9971 - batch_metric-mean_IoU: 0.8217 - batch_metric-mean_DSC: 0.6096
Epoch 00009: loss improved from 0.00505 to 0.00466, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_same_se/model.h5
54/54 [==============================] - 3s 51ms/step - loss: 0.0047 - accuracy: 0.9972 - batch_metric-mean_IoU: 0.8220 - batch_metri

53/54 [============================>.] - ETA: 0s - loss: 0.1538 - accuracy: 0.9681 - batch_metric-mean_IoU: 0.4245 - batch_metric-mean_DSC: 0.3488
Epoch 00001: loss improved from inf to 0.15151, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_valid_se/model.h5
54/54 [==============================] - 3s 52ms/step - loss: 0.1515 - accuracy: 0.9684 - batch_metric-mean_IoU: 0.4270 - batch_metric-mean_DSC: 0.3506 - val_loss: 0.0355 - val_accuracy: 0.9777 - val_batch_metric-mean_IoU: 0.5735 - val_batch_metric-mean_DSC: 0.4524 - lr: 0.0010
Epoch 2/10
53/54 [============================>.] - ETA: 0s - loss: 0.0153 - accuracy: 0.9904 - batch_metric-mean_IoU: 0.6459 - batch_metric-mean_DSC: 0.4933
Epoch 00002: loss improved from 0.15151 to 0.01524, saving model to /home/h4/stfo194b/scratch/attila/out/run-2/without_valid_se/model.h5
54/54 [==============================] - 2s 42ms/step - loss: 0.0152 - accuracy: 0.9904 - batch_metric-mean_IoU: 0.6467 - batch_metric-mean_DSC: 0

In [12]:
out_f = out_path / 'tables.tex'
create_tex_experiments(config, out_path, out_f)

=== run #1: loaded 8 results from /home/h4/stfo194b/scratch/attila/out/run-0/summary.json
creating .tex table for 8 experiments configurations
creating .tex table for 8 experiments results
=== run #2: loaded 8 results from /home/h4/stfo194b/scratch/attila/out/run-1/summary.json
creating .tex table for 8 experiments configurations
creating .tex table for 8 experiments results
=== run #3: loaded 8 results from /home/h4/stfo194b/scratch/attila/out/run-2/summary.json
creating .tex table for 8 experiments configurations
creating .tex table for 8 experiments results
creating .tex table for 3 runs
